In [ ]:
# @title <p>Essential Import
import os, shutil, json
from PIL import Image
from zipfile import ZipFile
import matplotlib.pyplot as plt
import numpy as np, pandas as pd, random as rd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# @title <p>Torch Essential Import
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from math import ceil

In [ ]:
base_model = [
    # expand_ratio, channels, repeats, stride, kernel_size
    [1, 16, 1, 1, 3],
    [6, 24, 2, 2, 3],
    [6, 40, 2, 2, 5],
    [6, 80, 3, 2, 3],
    [6, 112, 3, 1, 5],
    [6, 192, 4, 2, 5],
    [6, 320, 1, 1, 3],
]

phi_values = {
    # tuple of: (phi_value, resolution, drop_rate)
    "b0": (0, 224, 0.2),  # alpha, beta, gamma, depth = alpha ** phi
    "b1": (0.5, 240, 0.2),
    "b2": (1, 260, 0.3),
    "b3": (2, 300, 0.3),
    "b4": (3, 380, 0.4),
    "b5": (4, 456, 0.4),
    "b6": (5, 528, 0.5),
    "b7": (6, 600, 0.5),
}

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, stride, padding, groups=1):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding, groups=groups, bias=False)
        self.silu = nn.SiLU()
        self.bn = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        return self.silu(self.bn(self.conv(x)))

class SqueezeExcitation(nn.Module):
    def __init__(self, in_ch, red_dim):
        super().__init__()

        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_ch, red_dim, 1),
            nn.SiLU(),
            nn.Conv2d(red_dim, in_ch, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.se(x)

class InvertedResidual(nn.Module):
    def __init__(
        self,
        in_ch,
        out_ch,
        kernel_size,
        stride,
        padding,
        expand_ratio,
        reduction=4,
        survival_prob=0.8
    ):
        super().__init__()

        self.survival_prob = survival_prob
        self.use_residual = in_ch == out_ch and stride == 1
        hid_dim = in_ch * expand_ratio
        self.expand = in_ch != hid_dim
        red_dim = int(in_ch / reduction)

        if self.expand:
            self.expand_conv = ConvBlock(in_ch, hid_dim, kernel_size=3, stride=1, padding=1)

        self.conv = nn.Sequential(
            ConvBlock(hid_dim, hid_dim, kernel_size, stride, padding, groups=hid_dim),
            SqueezeExcitation(hid_dim, red_dim),
            nn.Conv2d(hid_dim, out_ch, 1, bias=False),
            nn.BatchNorm2d(out_ch)
        )

    def stochastic_depth(self, x):
        if not self.training:
            return x

        binary_tensor = (torch.rand(x.shape[0], 1, 1, 1, device=x.device) < self.survival_prob)
        return torch.div(x, self.survival_prob) * binary_tensor

    def forward(self, inputs):
        x = self.expand_conv(inputs) if self.expand else inputs
        if self.use_residual:
            return self.stochastic_depth(self.conv(x)) + inputs
        else:
            return self.conv(x)

class EfficientNet(nn.Module):
    def __init__(self, ver, num_classes):
        super().__init__()
        width_factor, depth_factor, dropout_rate = self.calculate_factors(ver)
        last_ch = int(1280 * width_factor)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.features = self.create_features(width_factor, depth_factor, last_ch)
        self.linear = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(last_ch, num_classes)
        )

    def calculate_factors(self, ver, alpha=1.2, beta=1.1):
        phi, res, drop_rate = phi_values[ver]
        depth_factor = alpha ** phi
        width_factor = beta ** phi
        return width_factor, depth_factor, drop_rate

    def create_features(self, width_factor, depth_factor, last_ch):
        channels = int(32 * width_factor)
        features = [ConvBlock(3, channels, 3, stride=2, padding=1)]
        in_ch = channels

        for expand_ratio, channels, repeats, stride, kernel_size in base_model:
            out_ch = int(4 * ceil(channels * width_factor / 4))
            layers_repeats = int(ceil(repeats * depth_factor))

            for layer in range(layers_repeats):
                features.append(
                    InvertedResidual(
                        in_ch,
                        out_ch,
                        expand_ratio=expand_ratio,
                        stride=stride if layer == 0 else 1,
                        kernel_size=kernel_size,
                        padding=kernel_size // 2
                    )
                )
                in_ch = out_ch

        features.append(
            ConvBlock(in_ch, last_ch, kernel_size=1, stride=1, padding=0)
        )

        return nn.Sequential(*features)

    def forward(self, x):
        x = self.pool(self.features(x))
        return self.linear(x.view(x.shape[0], -1))

In [ ]:
model = EfficientNet("b0", 1000)
x = torch.randn(1, 3, 224, 224)
out = model(x)
out.shape

torch.Size([1, 1000])